In [1]:
import sys
sys.path.append('/cellar/users/zkoch/methylation_and_mutation/source_files')
%load_ext autoreload
%aimport mutation_features, methylation_pred
%autoreload 1
import get_data, analysis, utils, plotting, compute_comethylation, methyl_mut_burden, somatic_mut_clock, mutation_features, methylation_pred

In [2]:
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
import pickle
import dask.dataframe as dd
from scipy.stats import spearmanr
from rich.progress import track
import statsmodels.formula.api as smf
import matplotlib.ticker as ticker
import colorcet as cc
from scipy.stats import spearmanr
from scipy import stats
import dask
import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV

%config InlineBackend.figure_format = 'retina'
plt.style.use("seaborn-deep")

In [45]:
out_dir = "/cellar/users/zkoch/methylation_and_mutation/output_dirs/021423_output"
dependency_f_dir = "./dependency_files"
data_dir = "./data"
corr_dir = '/cellar/users/zkoch/methylation_and_mutation/dependency_files/chr_dset_corrs'
#methylation_dir = '/cellar/users/zkoch/methylation_and_mutation/data/dropped3SD_qnormed_methylation'
methylation_dir = '/cellar/users/zkoch/methylation_and_mutation/data/processed_methylation'
icgc_dir = "/cellar/users/zkoch/methylation_and_mutation/data/icgc"

# TCGA

### Get data

In [4]:
illumina_cpg_locs_df, all_mut_df, all_methyl_df, all_methyl_df_t, all_meta_df, dataset_names_list = get_data.main(
    illum_cpg_locs_fn = os.path.join(dependency_f_dir, "illumina_cpg_450k_locations.csv"),
    out_dir = out_dir,
    methyl_dir = methylation_dir,
    mut_fn = os.path.join(data_dir, "PANCAN_mut.tsv.gz"),
    meta_fn = os.path.join(data_dir, "PANCAN_meta.tsv"))

Got mutations and metadata, reading methylation
Converting Dask df to pandas df, takes ~10min
Got methylation, transposing
Done


In [5]:
# read in other already computed files
mut_in_measured_cpg_w_methyl_age_df = pd.read_parquet(os.path.join(dependency_f_dir, "mut_in_measured_cpg_w_methyl_age_df_5year.parquet"))
all_mut_w_age_df, all_methyl_age_df_t = utils.add_ages_to_mut_and_methyl(all_mut_df, all_meta_df, all_methyl_df_t)

In [6]:
# read in train and test samples
train_samples_fn = os.path.join(dependency_f_dir, "train_samples.txt")
with open(train_samples_fn, "r") as f:
    training_samples = f.read().splitlines()
# read in train and test samples
test_samples_fn = os.path.join(dependency_f_dir,"test_samples.txt")
with open(test_samples_fn, "r") as f:
    testing_samples = f.read().splitlines()
# read in mi
mi_combined_df = pd.read_parquet(os.path.join(dependency_f_dir, 'mutual_informations', 'tcga_combinedMI_top10MI.parquet'))

In [7]:
skcm_training_samples = list(set(all_methyl_age_df_t[all_methyl_age_df_t['dataset'] == 'SKCM'].index.to_list()).intersection(set(training_samples)))
skcm_testing_samples = list(set(all_methyl_age_df_t[all_methyl_age_df_t['dataset'] == 'SKCM'].index.to_list()).intersection(set(testing_samples)))

# ICGC

### Get data

In [28]:
icgc_mut_df = pd.read_parquet("/cellar/users/zkoch/methylation_and_mutation/data/icgc/for_matrixQTL/icgc_mut_df.parquet")
icgc_meta_df = pd.read_parquet("/cellar/users/zkoch/methylation_and_mutation/data/icgc/for_matrixQTL/icgc_meta_df.parquet")
illumina_cpg_locs_df = get_data.get_illum_locs(os.path.join(dependency_f_dir, "illumina_cpg_450k_locations.csv"))
"""icgc_methyl_dd = dd.read_parquet('/cellar/users/zkoch/methylation_and_mutation/data/icgc/methyl_dir')
icgc_methyl_df = icgc_methyl_dd.compute()
icgc_methyl_df_t = icgc_methyl_df.T
shared_samples = set(icgc_methyl_df_t.index) & set(icgc_mut_df['sample'].unique()) & set(icgc_meta_df['sample'].unique())
icgc_methyl_df_t = icgc_methyl_df_t.loc[shared_samples]
icgc_methyl_df_t.dropna(how = 'any', axis=1, inplace=True)
icgc_mi_df = pd.read_parquet('/cellar/users/zkoch/methylation_and_mutation/output_dirs/011723_output/icgc_mi.parquet')
icgc_mi_df.sort_values(by='mutual_info', ascending=False, inplace=True)"""

"icgc_methyl_dd = dd.read_parquet('/cellar/users/zkoch/methylation_and_mutation/data/icgc/methyl_dir')\nicgc_methyl_df = icgc_methyl_dd.compute()\nicgc_methyl_df_t = icgc_methyl_df.T\nshared_samples = set(icgc_methyl_df_t.index) & set(icgc_mut_df['sample'].unique()) & set(icgc_meta_df['sample'].unique())\nicgc_methyl_df_t = icgc_methyl_df_t.loc[shared_samples]\nicgc_methyl_df_t.dropna(how = 'any', axis=1, inplace=True)\nicgc_mi_df = pd.read_parquet('/cellar/users/zkoch/methylation_and_mutation/output_dirs/011723_output/icgc_mi.parquet')\nicgc_mi_df.sort_values(by='mutual_info', ascending=False, inplace=True)"

In [30]:
icgc_meta_df['dataset'].value_counts()

PRAD-CA    241
PACA-AU    167
OV-AU       93
PBCA-DE     74
CLLE-ES     55
PAEN-AU     22
UTCA-FR     17
PRAD-UK      3
Name: dataset, dtype: int64

In [5]:
# rename columns 
icgc_mut_df.rename(columns={'chromosome':'chr', 'sample': 'case_submitter_id', 'chromosome_start':'start', 'MAF': 'DNA_VAF'}, inplace=True)
icgc_meta_df.rename(columns={'sample': 'case_submitter_id'}, inplace=True)
# merge with mut
icgc_mut_w_age_df = icgc_mut_df.merge(icgc_meta_df, on='case_submitter_id', how='left')
# and methyl dfs
icgc_meta_df_to_merge = icgc_meta_df[['case_submitter_id', 'age_at_index', 'dataset', 'gender']]
icgc_meta_df_to_merge.set_index('case_submitter_id', inplace=True)
# make gender column uppercase
icgc_meta_df_to_merge['gender'] = icgc_meta_df_to_merge['gender'].str.upper()
icgc_methyl_age_df_t = icgc_methyl_df_t.merge(icgc_meta_df_to_merge, left_index=True, right_index=True, how='left')

/tmp/ipykernel_2262854/2635062645.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icgc_meta_df_to_merge['gender'] = icgc_meta_df_to_merge['gender'].str.upper()


# TODO

In [ ]:
"""
1) Profile what the slowest part of creating feature matrices is and/or parallelize (either through bash jobs or python)
2) Wrap the whole thing in a 5 fold cross validation, so changing the training and testing samples each time
2) for some reason the training samples are not getting subset correctly
3) Get the ICGC data to work
"""

# Mutation features

In [46]:
mut_feat = mutation_features.mutationFeatures(
    all_mut_w_age_df = all_mut_w_age_df, illumina_cpg_locs_df = illumina_cpg_locs_df, 
    all_methyl_age_df_t = all_methyl_age_df_t, out_dir = out_dir, consortium = 'TCGA', dataset = 'SKCM',
    cross_val_num = 1,
    matrix_qtl_dir = "/cellar/users/zkoch/methylation_and_mutation/data/matrixQtl_data/muts"
    )

In [48]:
# choose which CpGs to generate features for
skcm_mi_df = mi_combined_df['SKCM'].to_frame().rename(columns = {'SKCM':'mutual_info'})
cpg_pred_priority = mut_feat.choose_cpgs_to_train(mi_df = skcm_mi_df, bin_size=20000, sort_by=['mutual_info', 'count'])
mut_feat.create_all_feat_mats(
    cpg_ids = cpg_pred_priority.head(2)['#id'].to_list(), aggregate='Both',
    num_correl_sites=5000, num_correl_ext_sites=100, max_meqtl_sites=100,
    nearby_window_size=25000
    )
mut_feat_store_fn = mut_feat.save_mutation_features()

Finished 0 of 2
Saved mutation features to
/cellar/users/zkoch/methylation_and_mutation/output_dirs/021423_output/TCGA_SKCM_5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2numCpGs_startTopCpGs_crossValNum/TCGA_SKCM_5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2numCpGs_startTopCpGs_crossValNum.features.pkl


In [52]:
mut_feat_store_fns = [mut_feat_store_fn]
# train predictors for these cpgs
methyl_pred = methylation_pred.methylationPrediction(
    mut_feat_store_fns = mut_feat_store_fns,
    model_type = 'xgboost'
    )
methyl_pred.train_all_models()
methyl_pred.apply_all_models()
methyl_pred.save_models_and_preds()

done 0 CpGs of 2
Predicted methylation for 0 CpGs of 2


In [55]:
methyl_pred.perf_df

,cg16622920,cg11793449
r2,0.000957,0.009384
mae,0.191055,0.102767


In [65]:
with open("/cellar/users/zkoch/methylation_and_mutation/output_dirs/020723_output/TCGA_SKCM_5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2500numCpGs_2500startTopCpGs_0crossValNum/TCGA_SKCM_5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2500numCpGs_2500startTopCpGs_0crossValNum.features.pkl", 'rb') as f:
        mut_feat_store = pickle.load(f)

In [15]:
methyl_pred_perf_df = pd.read_parquet("/cellar/users/zkoch/methylation_and_mutation/output_dirs/020723_output/TCGA_SKCM_5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2500numCpGs_0startTopCpGs_0crossValNum/prediction_performance_xgboost.parquet")

In [25]:
methyl_pred_perf_df.loc['r2'].sort_values(ascending=False).head(20)

cg09807229    0.932512
cg22736444    0.850674
cg16314017    0.812843
cg25949513    0.789752
cg11076862    0.771992
cg11994630    0.764812
cg08858662    0.721128
cg11875773    0.718832
cg23847185    0.684714
cg22304169    0.684339
cg27403822    0.680632
cg06948825    0.668372
cg03347632    0.657688
cg19642494    0.653494
cg17885806    0.639160
cg09540111    0.629850
cg23991039    0.621726
cg13403223    0.574650
cg05037168    0.561266
cg18861300    0.555698
Name: r2, dtype: float64

# ICGC testing

In [230]:
icgc_mut_feat = mutation_features.mutationFeatures(
    all_mut_w_age_df = icgc_mut_w_age_df, illumina_cpg_locs_df = illumina_cpg_locs_df, 
    all_methyl_age_df_t = icgc_methyl_age_df_t, out_dir = out_dir, dataset = 'PRAD-CA',
    consortium = 'ICGC', train_samples = train, test_samples = test,
    matrix_qtl_dir = "/cellar/users/zkoch/methylation_and_mutation/output_dirs/icgc_muts_011423"
    )

In [289]:
icgc_cpg_pred_priority = icgc_mut_feat.choose_cpgs_to_train(mi_df = icgc_mi_df, bin_size=20000, sort_by=['mutual_info', 'count'])
icgc_mut_feat.create_all_feat_mats(
    cpg_ids = icgc_cpg_pred_priority.head(10)['#id'].to_list(), aggregate='Both',
    num_correl_sites=5000, num_correl_ext_sites=100, max_meqtl_sites=100,
    nearby_window_size=25000
    )

Finished 0 of 10


In [ ]:
mut_feat_store_fns = glob.glob('/cellar/users/zkoch/methylation_and_mutation/output_dirs/020723_output/ICGC__5000correl_100correlExt_100meqtl_25000nearby_Bothagg_2500numCpGs_*/*features.pkl')
# train predictors for these cpgs
icgc_methyl_pred = methylation_pred.methylationPrediction(
    mut_feat_store_fns = mut_feat_store_fns,
    model_type = 'xgboost'
    )
icgc_methyl_pred.train_all_models()
icgc_methyl_pred.apply_all_models()
icgc_methyl_pred.save_models_and_preds()

In [75]:
icgc_methyl_pred.pred_df

,cg21939836,cg19513004,cg10611580,cg19765450,cg25738301,cg20324262,cg12198841,cg14858014,cg11023456,cg20318272,...,cg25765464,cg03866192,cg24289912,cg21265427,cg24432193,cg06780032,cg19862213,cg12798157,cg26050734,cg11677683
DO229587,0.816891,0.825813,0.474730,0.703935,0.711469,0.168689,0.614270,0.701690,0.586329,0.479562,...,0.466744,0.610034,0.462205,0.829533,0.674039,0.644141,0.710154,0.523064,0.627139,0.767209
DO34264,0.709653,0.699921,0.209666,0.662381,0.707842,0.173406,0.707967,0.628674,0.571434,0.681396,...,0.718385,0.238607,0.367543,0.835807,0.331577,0.236527,0.778839,0.329154,0.635916,0.795738
DO6492,0.840957,0.308044,0.119956,0.719026,0.833976,0.286467,0.802153,0.755499,0.899448,0.692306,...,0.785762,0.098763,0.090268,0.878880,0.228655,0.183940,0.851975,0.255794,0.496408,0.916266
DO51050,0.816891,0.825813,0.474730,0.703935,0.711469,0.168689,0.586204,0.732861,0.586329,0.469216,...,0.455235,0.610034,0.396187,0.829533,0.674039,0.644141,0.710154,0.523064,0.627139,0.814853
DO32860,0.730505,0.705514,0.235543,0.646740,0.717188,0.180174,0.669995,0.593576,0.568219,0.598016,...,0.730566,0.261108,0.520969,0.827235,0.423808,0.288732,0.754906,0.371720,0.632526,0.756701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DO6436,0.835258,0.325335,0.107240,0.710940,0.812999,0.236781,0.821971,0.779654,0.896851,0.727753,...,0.800279,0.092550,0.090425,0.886819,0.156284,0.231390,0.869960,0.197517,0.435292,0.899429
DO230505,0.816891,0.825813,0.474730,0.703935,0.711469,0.168689,0.614270,0.701690,0.586329,0.487908,...,0.466744,0.610034,0.462205,0.829533,0.674039,0.644141,0.710154,0.523064,0.627139,0.767209
DO46528,0.765537,0.664663,0.276289,0.698257,0.454159,0.151031,0.574600,0.443121,0.539148,0.807248,...,0.399743,0.502328,0.148001,0.867028,0.370777,0.161262,0.829850,0.309106,0.434976,0.841880
DO230521,0.816891,0.825813,0.474730,0.703935,0.711469,0.168689,0.614270,0.701690,0.586329,0.479562,...,0.466744,0.610034,0.421031,0.829533,0.674039,0.644141,0.710154,0.523064,0.627139,0.421442
